In [1]:
# input : 시 73:24~26
# process
# 입력 데이타 의미해석을 위해 분리
#먼저
# paragraph_id = '시 73:24~26'
# paragraph_id = '롬 11:36'
paragraph_id = " 빌 2:5∼11"
bible_chapter_section = { 'bible_name' : "",
                       'chapter' : "",
                       'from_section' : "",
                       'to_section' : ""
                       }

# bible_chapter_section = {}

first_processing_list = paragraph_id.strip().split(' ')
bible_chapter_section['bible_name'] = first_processing_list[0]
print("성경과장절분리:",first_processing_list)

second_processing_list = first_processing_list[1].split(':')
bible_chapter_section['chapter'] = second_processing_list[0]

print("장과절분리:",second_processing_list)
print("장:",second_processing_list[0])
print("절:",second_processing_list[1])
print("절 분리:",second_processing_list[1].split('∼'))
print("절 분리('∼'):",second_processing_list[1].split('∼'))

third_processing_list=[]
if (('~') in second_processing_list[1]):
    third_processing_list = second_processing_list[1].split('~')
    print("시작절:",third_processing_list[0])
    print("마지막절:",third_processing_list[1])
    bible_chapter_section['from_section'] = third_processing_list[0]
    bible_chapter_section['to_section'] = third_processing_list[1]
elif (('∼') in second_processing_list[1]):
    third_processing_list = second_processing_list[1].split('∼')
    print("시작절:",third_processing_list[0])
    print("마지막절:",third_processing_list[1])
    bible_chapter_section['from_section'] = third_processing_list[0]
    bible_chapter_section['to_section'] = third_processing_list[1]   
else:
    bible_chapter_section['from_section'] = second_processing_list[1]
    
print(third_processing_list)
    
print(bible_chapter_section)


성경과장절분리: ['빌', '2:5∼11']
장과절분리: ['2', '5∼11']
장: 2
절: 5∼11
절 분리: ['5', '11']
절 분리('∼'): ['5', '11']
시작절: 5
마지막절: 11
['5', '11']
{'bible_name': '빌', 'chapter': '2', 'from_section': '5', 'to_section': '11'}


In [2]:
# 성경약어로 대한성서공회 사용 성경영문약어를 가져와서 성경구절 질의 정보를 완성
import json

query_info = {
    "bible_name" : "",   # 대한성서공회에서 사용한 성경 영문약어명
    "chapter" : "",      # 성경 장
    "from_section" : "", # 시작구절
    "to_section" : "",   # 종료구절
}

# 요청한 성경 약어 획득
bible_name = bible_chapter_section['bible_name']
print("bible_name:%s" % bible_name)

# 요청 성경 약어로부터 대한성서공회 성경영문약어를 정의한 JSON파일을 읽어서 성경영문약어 획득
with open('data/catechism/bible_abbr.json', 'rt', encoding='UTF8') as json_data:
    biblical_abbreviation = json.load(json_data)

bible_name_info = biblical_abbreviation[bible_name]
#print(bible_name_info)
print("bskorea__name:%s" % bible_name_info['bskorea__name'])

# 성경구절 질의 정보 생성
chapter = bible_chapter_section['chapter']
print("chapter:%s" % chapter)

query_info['bible_name'] = bible_name_info['bskorea__name']
query_info['chapter'] = bible_chapter_section['chapter']
query_info['from_section'] = bible_chapter_section['from_section']
query_info['to_section'] = bible_chapter_section['to_section']

print("query_info", query_info)


bible_name:빌
bskorea__name:php
chapter:2
query_info {'bible_name': 'php', 'chapter': '2', 'from_section': '5', 'to_section': '11'}


In [3]:
import requests
from bs4 import BeautifulSoup

# 대한성서공회
# url setting
version = 'HAN'         # 개역한글
book = 'gen'            # 창세기
chapter = 1             # 1장을 가져온다.

# 목표 url
#     url = "http://www.bskorea.or.kr/bible/korbibReadpage.php?version=GAE&book=gen&chap=1"
# 웹 서버에 요청 보내고 응답 받기
resp = requests.get("https://www.bskorea.or.kr/bible/korbibReadpage.php?version=HAN&book={}&chap={}".format(query_info['bible_name'], query_info['chapter']), verify=False)
# 전송된 HTML 문서 얻기
html = resp.text
# HTML 문서를 DOM으로 변환하기
soup = BeautifulSoup(html, "lxml")
# 목표 HTML 요소 선택(DIV tag중 ID가 tdBible1인 요소의 하위에 SPAN tag 모두)
html_element = soup.find("div", {"id" : "tdBible1"}).find_all("span")
# print(html_element)

# 목표 텍스트 추출
bible_chapter_table = {}
print("========================")
for bible_chapter_verses in html_element:
    # HTML DIV tag중 ID가 tdBible1인 요소의 하위에 SPAN tag 모두를 가지고 와서
    # SPAN tag의 텍스트 성경 구절들을 List객체로 만듬
    print("========================")
    # List객체중 " \xa0\xa0\xa0"이 없는 내용은 버림
    if "xa0\xa0\xa0" in bible_chapter_verses:
        continue

#     print("bible_chapter_verses : ", bible_chapter_verses)
    table0 = bible_chapter_verses.get_text().strip().split("  ")
#     print("table0 : ", table0)
        
    # List객체중 "\xa0\xa0\xa0"로 좌우 분리
    table1 = table0[0].split("\xa0\xa0\xa0")
#     print("table1 : ", table1)

    # List객체중 구절 번호와 내용의 쌍으로 된것만 선택
    if len(table1) > 1:
        # List객체중 ""\n\n"가 들어 있는것을 좌우 분리하여 죄측내용만 선택
        tmp = table1[1].rsplit("\n\n")
        if len(tmp) > 1:
            verses = tmp[0]
        else:
            verses = table1[1]

        bible_chapter_table[table1[0]] = verses
        print("table1==>",table1)

print("gen_table==>",bible_chapter_table)
print("gen_table['2']==>",bible_chapter_table['2'])

print('Complete!\n')

table1==> ['1', '그러므로 그리스도 안에 무슨 권면이나 사랑에 무슨 위로나 성령의 무슨 교제나 긍휼이나 자비가 있거든']
table1==> ['2', '마음을 같이 하여 같은 사랑을 가지고 뜻을 합하며 한 마음을 품어']
table1==> ['3', '아무 일에든지 다툼이나 허영으로 하지 말고 오직 겸손한 마음으로 각각 자기보다 남을 낫게 여기고']
table1==> ['4', '각각 자기 일을 돌아볼 뿐더러 또한 각각 다른 사람들의 일을 돌아보아 나의 기쁨을 충만케 하라']
table1==> ['5', '너희 안에 이 마음을 품으라 곧 그리스도 예수의 마음이니']
table1==> ['6', '그는 근본 하나님의 1)본체시나 하나님과 동등됨을 2)취할 것으로 여기지 아니하시고 \n\n\n\n혹 형체 혹 보류']
table1==> ['7', '오히려 자기를 비워 종의 3)형체를 가져 사람들과 같이 되었고 \n\n혹 본체']
table1==> ['8', '사람의 모양으로 나타나셨으매 자기를 낮추시고 죽기까지 복종하셨으니 곧 십자가에 죽으심이라']
table1==> ['9', '이러므로 하나님이 그를 지극히 높여 모든 이름 위에 뛰어난 이름을 주사']
table1==> ['10', '하늘에 있는 자들과 땅에 있는 자들과 땅 아래 있는 자들로 모든 무릎을 예수의 이름에 꿇게 하시고']
table1==> ['11', '모든 입으로 예수 그리스도를 주라 시인하여 하나님 아버지께 영광을 돌리게 하셨느니라']
table1==> ['12', '그러므로 나의 사랑하는 자들아 너희가 나 있을 때 뿐아니라 더욱 지금 나 없을 때에도 항상 복종하여 두렵고 떨림으로 너희 구원을 이루라']
table1==> ['13', '너희 안에서 행하시는 이는 하나님이시니 자기의 기쁘신 뜻을 위하여 너희로 소원을 두고 행하게 하시나니']
table1==> ['14', '모든 일을 원망과 시비가 없이 하라']
table1==> ['15', '이는 너희가 흠이 없고 순전하여 어그러지고 거

C:\Users\saint\ChatBot\chat_env\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
# 성경구절 획득하여 Return
response_value = ""
                 
if query_info['to_section']:
    start_verses = int(query_info['from_section'])
    end_verses = int(query_info['to_section'])
    for i in range(start_verses, end_verses+1):
        print("str(i)[{}]".format(str(i)))
        response_value += " "
        response_value += str(i)
        response_value += ". "
        response_value += bible_chapter_table[str(i)]
else:
    response_value += query_info['from_section']
    response_value += ". "
    response_value += bible_chapter_table[query_info['from_section']]
    
print("response_value : [%s]" % response_value.strip())

str(i)[5]
str(i)[6]
str(i)[7]
str(i)[8]
str(i)[9]
str(i)[10]
str(i)[11]
response_value : [5. 너희 안에 이 마음을 품으라 곧 그리스도 예수의 마음이니 6. 그는 근본 하나님의 1)본체시나 하나님과 동등됨을 2)취할 것으로 여기지 아니하시고  7. 오히려 자기를 비워 종의 3)형체를 가져 사람들과 같이 되었고  8. 사람의 모양으로 나타나셨으매 자기를 낮추시고 죽기까지 복종하셨으니 곧 십자가에 죽으심이라 9. 이러므로 하나님이 그를 지극히 높여 모든 이름 위에 뛰어난 이름을 주사 10. 하늘에 있는 자들과 땅에 있는 자들과 땅 아래 있는 자들로 모든 무릎을 예수의 이름에 꿇게 하시고 11. 모든 입으로 예수 그리스도를 주라 시인하여 하나님 아버지께 영광을 돌리게 하셨느니라]
